In [1]:
import pandas as pd
import yaml

pd.set_option('display.max_columns', None)

with open('../../Settings.yaml', 'r') as file:
    Setting = yaml.safe_load(file)


    
file_name = 'PPI.xlsx'
file_path = f"{Setting['Raw_Path']}/{file_name}"
PPI_Dataset = pd.read_excel(file_path, sheet_name=6 ,skiprows=2)
PPI_Dataset = PPI_Dataset.iloc[1:,1:]
PPI_Dataset.columns.values[0] = 'Industry_Name'
New_Row_Value = PPI_Dataset.iloc[0, 1:].tolist()

#Adding new row because PPI dataset does not have value for 'تعمير ونصب ماشين آلات' industry 
#And fullfill it with averge PPI of Whole Industry
New_Row = pd.DataFrame([['تعمير ونصب ماشين آلات'] + New_Row_Value], columns=PPI_Dataset.columns)
PPI_Dataset = pd.concat([PPI_Dataset, New_Row], ignore_index=True)
PPI_Dataset["Industry_Code"] = ['پ'] + list(range(10, 9 + len(PPI_Dataset)))

#Reordering columns
cols = PPI_Dataset.columns.tolist()
cols.remove("Industry_Code")
new_order = ["Industry_Code"] + cols
PPI_Dataset = PPI_Dataset[new_order]
PPI_Dataset.drop(columns=['Industry_Name'], inplace=True)

In [2]:
input_file = f'{Setting['Workshop_Codes']}'
Workshop_Codes = pd.read_excel(input_file)
Workshop_Codes = Workshop_Codes[Workshop_Codes['Industry_Category_Code'].isin([1,2])]

In [3]:
PPI_Dataset['Industry_Code'] = PPI_Dataset['Industry_Code'].astype(str)
Workshop_Codes['Industry_Code'] = Workshop_Codes['Industry_Code'].astype(str)

In [4]:
PPI_Dataset = pd.merge(PPI_Dataset, Workshop_Codes, on='Industry_Code', how='left')
cols = list(PPI_Dataset.columns)
cols.remove('Industry_Category_Code')
cols.remove('Industry_Name')
cols.insert(0, 'Industry_Category_Code') 
cols.insert(2, 'Industry_Name')       
PPI_Dataset = PPI_Dataset[cols]

In [5]:
#Reshaping Dataset
PPI_Dataset = pd.melt(PPI_Dataset, 
                   id_vars=PPI_Dataset.columns[:3], 
                   value_vars=PPI_Dataset.columns[3:], 
                   var_name='Year', 
                   value_name='Price_Index')

#Reordering columns
cols = list(PPI_Dataset.columns)
cols.remove('Year')
cols.insert(0, 'Year')        
PPI_Dataset = PPI_Dataset[cols]

In [6]:
output_file_name = '002_PPI.xlsx'
PPI_Dataset.to_excel(f"{Setting['Output_Path_General']}/{output_file_name}", index=False)
